# Analysis of quasi-elastic neutron scattering (QENS) measurement of a protein in liquid state

This file contains the code of the nPDyn documentation available here: https://npdyn.readthedocs.io/en/latest/.

In [1]:
import numpy as np

from nPDyn import Dataset


%matplotlib inline
import matplotlib.pyplot as plt


In [2]:
qens = Dataset(
    QENSFiles=['sample_data/lys_part_01_QENS_before_280K.nxs'],
    resFiles=['sample_data/vana_QENS_280K.nxs'],
    ECFile='sample_data/empty_cell_QENS_280K.nxs',
    D2OFile='sample_data/D2O_QENS_280K.nxs')

## Process the data

In [3]:
qens.binAll(5)
qens.subtract_EC()
qens.setQRange(0.4, 1.8)
q = qens.dataList[0].data.qVals[:, np.newaxis]
X = qens.dataList[0].data.energies

## Fit the resolution and D2O background

In [4]:
from nPDyn.lmfit.lmfit_presets import pseudo_voigt, calibratedD2O

qens.fitRes(model=pseudo_voigt(q, prefix='res_'), cleanData='omit')
qens.D2OData.fit(model=calibratedD2O(q, 0.95, 280, prefix='D2O_'), cleanData='omit')

Fitting dataset: sample_data/vana_QENS_280K.nxs


/home/kpounot/.local/lib/python3.6/site-packages/nPDyn-2.0+0.g9f9576f.dirty-py3.6-linux-x86_64.egg/nPDyn/dataTypes/baseType.py:673: RuntimeWarning: divide by zero encountered in true_divide
  errors = 1 / errors



Done.

Fitting dataset: sample_data/D2O_QENS_280K.nxs
	Fit of observable 1 of 1 (time=0.0)
Done.



## Build the model

In [5]:
from nPDyn.lmfit.lmfit_presets import build_2D_model

# let us start with the formatted text for the center-of-mass term.
comText = ("{beta} * {alpha} * {Ds} * {q}**2 / (np.pi * "
           "(x**2 + ({Ds} * {q}**2)**2))")

# same for the internal dynamics term
jumpDiffText = ("{beta} * (1 - {alpha}) * "
                "({Di} * {q}**2 / (1 + {Di} * {q}**2 * {tau})) / "
                "(np.pi * (x**2 + ({Di} * {q}**2 / "
                "(1 + {Di} * {q}**2 * {tau}))**2))")

# now we build the components
comModel = build_2D_model(
    q,
    'com',
    comText,
    paramGlobals=['alpha', 'Ds'],
    bounds={
        'beta': (0., np.inf),
        'alpha': (0, 1),
        'Ds': (0.001, np.inf)},  # non-zero minimum to avoid infinite values
    defVals={'alpha': 0.5,
             'Ds': 5,
             'beta': 1},
    prefix='com_')

jumpDiffModel = build_2D_model(
    q,
    'jumpDiff',
    jumpDiffText,
    paramGlobals=['alpha', 'Di', 'tau'],
    bounds={
        'beta': (0., np.inf),
        'alpha': (0, 1),
        'Di': (0.001, np.inf),  # non-zero minimum to avoid infinite values
        'tau': (0., np.inf)},
    defVals={'beta': 1,
             'alpha': 0.5,
             'Di': 30,
             'tau': 10},
    prefix='jd_')

# and we assemble them
model = comModel + jumpDiffModel

# some parameters are the same for the two components, so we set them equals using 'expr' hint
model.set_param_hint('com_alpha', expr='jd_alpha')
for i in range(q.size):
    model.set_param_hint('com_beta_%i' % i, expr='jd_beta_%i' % i)

In [6]:
# now we add the component for the D2O signal
from nPDyn.lmfit.lmfit_presets import hline

scale = hline(q, prefix='bD2O_', bounds={'b': (0.0001, np.inf)})
d2OModel = scale * qens.D2OData.model
d2OModel.param_hints.update(qens.D2OData.getFixedOptParams(0))

fitModel = model + d2OModel

## Fit the data

In [7]:
qens.fitData(model=fitModel, q=q, convolveRes=True, cleanData='omit')

Fitting dataset: sample_data/lys_part_01_QENS_before_280K.nxs


/home/kpounot/.local/lib/python3.6/site-packages/nPDyn-2.0+0.g9f9576f.dirty-py3.6-linux-x86_64.egg/nPDyn/lmfit/lmfit_presets.py:3: RuntimeWarning: invalid value encountered in true_divide
  
/home/kpounot/.local/lib/python3.6/site-packages/nPDyn-2.0+0.g9f9576f.dirty-py3.6-linux-x86_64.egg/nPDyn/lmfit/lmfit_presets.py:4: RuntimeWarning: invalid value encountered in true_divide
  These functions generate a `Model` class instance from
/home/kpounot/.local/lib/python3.6/site-packages/nPDyn-2.0+0.g9f9576f.dirty-py3.6-linux-x86_64.egg/nPDyn/lmfit/lmfit_presets.py:5: RuntimeWarning: invalid value encountered in true_divide
  the **lmfit** package [#]_.
/home/kpounot/.local/lib/python3.6/site-packages/nPDyn-2.0+0.g9f9576f.dirty-py3.6-linux-x86_64.egg/nPDyn/lmfit/lmfit_presets.py:6: RuntimeWarning: invalid value encountered in true_divide
  
/home/kpounot/.local/lib/python3.6/site-packages/nPDyn-2.0+0.g9f9576f.dirty-py3.6-linux-x86_64.egg/nPDyn/lmfit/lmfit_presets.py:7: RuntimeWarning: invalid 


Done.



In [8]:
qens.plotQENS()

/home/kpounot/.local/lib/python3.6/site-packages/nPDyn-2.0+0.g9f9576f.dirty-py3.6-linux-x86_64.egg/nPDyn/plot/qensPlot.py:213: UserWarning: Tight layout not applied. The left and right margins cannot be made large enough to accommodate all axes decorations. 
  self.canvas.draw()
/home/kpounot/.local/lib/python3.6/site-packages/nPDyn-2.0+0.g9f9576f.dirty-py3.6-linux-x86_64.egg/nPDyn/plot/qensPlot.py:213: UserWarning: Tight layout not applied. The left and right margins cannot be made large enough to accommodate all axes decorations. 
  self.canvas.draw()
/home/kpounot/.local/lib/python3.6/site-packages/nPDyn-2.0+0.g9f9576f.dirty-py3.6-linux-x86_64.egg/nPDyn/plot/qensPlot.py:334: UserWarning: Tight layout not applied. tight_layout cannot make axes width small enough to accommodate all axes decorations
  self.canvas.draw()
/home/kpounot/.local/lib/python3.6/site-packages/nPDyn-2.0+0.g9f9576f.dirty-py3.6-linux-x86_64.egg/nPDyn/plot/create_window.py:27: UserWarning: Tight layout not applied